In [5]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.8 MB/s eta 0:00:00


In [8]:
# Load dataset from uploaded file
dataset = load_dataset("text", data_files={"train": file_path})

Generating train split: 0 examples [00:00, ? examples/s]

In [9]:
# Load GPT-2 tokenizer and model
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [6]:
import torch
import transformers
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset
from google.colab import files

# Upload file
uploaded = files.upload()

# Get uploaded file name dynamically
file_name = list(uploaded.keys())[0]
file_path = f"/content/{file_name}"

# Load dataset from uploaded file
dataset = load_dataset("text", data_files={"train": file_path})

# Load GPT-2 tokenizer and model
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Set padding token
tokenizer.pad_token = tokenizer.eos_token

# Tokenization function (with labels)
def tokenize_function(examples):
    tokenized_output = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )
    tokenized_output["labels"] = tokenized_output["input_ids"].copy()  # Set labels same as input_ids
    return tokenized_output

# Tokenize dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["text"])  # Keep only tokenized inputs

# Split dataset into training and validation (90% train, 10% validation)
train_test_split = tokenized_datasets["train"].train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

# Define TrainingArguments
training_args = TrainingArguments(
    output_dir="./gpt2-story-model",
    evaluation_strategy="epoch",
    per_device_train_batch_size=4,  # Increase batch size if GPU allows
    per_device_eval_batch_size=4,
    num_train_epochs=5,  # Increase for better fine-tuning
    save_strategy="epoch",
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=100,
    report_to="none",
    fp16=torch.cuda.is_available(),  # Use mixed precision if available
    remove_unused_columns=False,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Train model
trainer.train()


Saving story.txt to story.txt


Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,No log,5.961102
2,No log,5.961102
3,No log,5.518859
4,No log,4.143492
5,No log,3.059378


TrainOutput(global_step=15, training_loss=6.548408508300781, metrics={'train_runtime': 46.4297, 'train_samples_per_second': 0.969, 'train_steps_per_second': 0.323, 'total_flos': 11758141440000.0, 'train_loss': 6.548408508300781, 'epoch': 5.0})

In [7]:
# Save model and tokenizer
model.save_pretrained("./gpt2-story-model")
tokenizer.save_pretrained("./gpt2-story-model")

# Zip the trained model
!zip -r gpt2-story-model.zip ./gpt2-story-model

  adding: gpt2-story-model/ (stored 0%)
  adding: gpt2-story-model/generation_config.json (deflated 24%)
  adding: gpt2-story-model/tokenizer_config.json (deflated 56%)
  adding: gpt2-story-model/vocab.json (deflated 68%)
  adding: gpt2-story-model/model.safetensors (deflated 7%)
  adding: gpt2-story-model/config.json (deflated 51%)
  adding: gpt2-story-model/merges.txt (deflated 53%)
  adding: gpt2-story-model/checkpoint-15/ (stored 0%)
  adding: gpt2-story-model/checkpoint-15/generation_config.json (deflated 24%)
  adding: gpt2-story-model/checkpoint-15/scheduler.pt (deflated 56%)
  adding: gpt2-story-model/checkpoint-15/model.safetensors (deflated 7%)
  adding: gpt2-story-model/checkpoint-15/optimizer.pt (deflated 7%)
  adding: gpt2-story-model/checkpoint-15/config.json (deflated 51%)
  adding: gpt2-story-model/checkpoint-15/trainer_state.json (deflated 68%)
  adding: gpt2-story-model/checkpoint-15/training_args.bin (deflated 52%)
  adding: gpt2-story-model/checkpoint-15/rng_state.p

In [8]:
# Download model
files.download("gpt2-story-model.zip")

# Load fine-tuned model for inference
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2-story-model")
model = GPT2LMHeadModel.from_pretrained("./gpt2-story-model")

def generate_story(prompt, max_length=200):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(
        input_ids,
        max_length=max_length,
        temperature=0.7,  # Lower to reduce hallucination
        top_p=0.8,  # Focus on high-probability words
        do_sample=True
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
# Example usage
prompt = "Once upon a time in a distant kingdom,"
story = generate_story(prompt)
print(story)

def ask_question(question, context, max_length=50):
    input_text = f"Context: {context}\nQuestion: {question}\nAnswer:"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    output = model.generate(
        input_ids,
        max_length=max_length,
        temperature=0.5,  # Lower temp for factual answers
        top_p=0.7,
        do_sample=True
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Once upon a time in a distant kingdom, there was a man named Jekyll, who had been a slave to the Lord, who had been born to the Lord, and now lived to the end. He was called Jekyll.

Jekyll is said to have lived to the end, and he was born to the Lord.


In [10]:
from google.colab import files

# Upload the text file manually in Colab
uploaded = files.upload()

# Read the uploaded story
story_filename = list(uploaded.keys())[0]  # Automatically gets the uploaded file name
with open(story_filename, "r", encoding="utf-8") as f:
    story_text = f.read()

print("Story Loaded Successfully!")


Saving story.txt to story (1).txt
Story Loaded Successfully!


In [11]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# Load fine-tuned model
model_path = "./gpt2-story-model"
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path).to("cuda" if torch.cuda.is_available() else "cpu")

print("Fine-tuned model loaded successfully!")


Fine-tuned model loaded successfully!


In [18]:
def ask_question(question, context, max_new_tokens=50):
    prompt = f"Story: {context}\nQuestion: {question}\nAnswer:"

    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
    output = model.generate(
        input_ids,
        max_new_tokens=max_new_tokens,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    # Debug: Print the raw output before processing
    raw_output = tokenizer.decode(output[0], skip_special_tokens=True)
    print("RAW OUTPUT:\n", raw_output)  # Debugging step

    return raw_output.split("Answer:")[-1].strip()

# Example run
question = "What did Emma Loose?"
answer = ask_question(question, story_text)

print(f"Q: {question}\nA: {answer}")


RAW OUTPUT:
 Story: The Lost Key
It was a stormy evening when Emma arrived home from work, drenched from the unexpected rain. She fumbled through her purse, searching for the familiar metallic feel of her house key. But it wasn’t there. A chill ran down her spine—not from the cold, but from the realization that she might have lost it. Her mind raced back to the café where she had stopped for coffee, the bus she had hurried onto, and the moment she had reached for her phone at the bookstore. Any of those places could be where she had left it.

Determined not to panic, Emma pulled out her phone and called her best friend, Jake. "I think I lost my house key," she admitted, her voice edged with worry. Jake, always the problem solver, suggested retracing her steps. Without hesitation, Emma ran through the now-empty streets, her shoes splashing in puddles as she made her way back to the bookstore. The cashier shook his head—no keys had been found. The café was closing, and the barista hadn't